<a href="https://colab.research.google.com/github/jeffufpost/stochastic_SEIR_model/blob/master/SEIR_comparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pycountry

     |████████████████████████████████| 10.0MB 7.7MB/s 
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627361 sha256=0978d315a0dca9201b88046a7ff8acfb4e4235a02bbbd815dc7e2810a3fc9e76
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry


In [0]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import gamma
from scipy.stats import poisson

import pycountry as pc

import statsmodels

## SEIR model stuff

In [0]:
def make_df(p,num_I):
  df = pd.DataFrame(np.full((p,1), 'S').T[0], columns=['State'])
  df['Day'] = 0
  df.loc[np.random.randint(1,p,num_I),'State'] = 'I'
  return df

In [0]:
def SEIR_model(r, beta, beta2, p, num_I, days, g1=None, g2=None):

  df = make_df(p,num_I)

  S=np.array([],dtype=int)
  E=np.array([],dtype=int)
  I=np.array([],dtype=int)
  R=np.array([],dtype=int)
  Cases=np.array([],dtype=int)

  Epd=np.array([],dtype=int)
  Ipd=np.array([],dtype=int)
  Rpd=np.array([],dtype=int)
  Casespd=np.array([],dtype=int)

  if g1 is None:
    g1=gamma.cdf(np.arange(days),2,loc=2.5,scale=2.5)
  else:
    g1 = g1
    
  if g2 is None:
    g2=gamma.cdf(np.arange(days),7,loc=4,scale=1)
  else:
    g2 = g2

  b=beta

  rand = np.random.random(size=(p,days))

  maskg1 = (rand[:,:] < g1[:])
  maskg2 = (rand[:,:] < g2[:])

  maskday1 = maskg1.argmax(1)
  maskday2 = maskg2.argmax(1)

  for j in range(0,days-1):

    S=np.append(S,len(np.where(df.State=='S')[0]))
    E=np.append(E,len(np.where(df.State=='E')[0]))
    I=np.append(I,len(np.where(df.State=='I')[0]))
    R=np.append(R,len(np.where(df.State=='R')[0]))

    Rpd=np.append(Rpd,len(df.loc[(df.State == 'I') & (j-df.Day >= maskday2)]))
    Ipd=np.append(Ipd,len(df.loc[(df.State == 'E') & (j-df.Day >= maskday1)]))
    Epd=np.append(Epd,len(df.loc[(df.State == 'S') & (rand[:,j] < r*b*len(np.where(df.State=='I')[0])/p)]))
    
    df.loc[(df.State == 'I') & (j-df.Day >= maskday2)] = ['R', j]
    df.loc[(df.State == 'E') & (j-df.Day >= maskday1)] = ['I', j]
    df.loc[(df.State == 'S') & (rand[:,j] < r*b*len(np.where(df.State=='I')[0])/p)] = ['E', j]
                            
    if Ipd.cumsum()[-1] > (p*0.1): b = beta2
#    elif Ipd[-1] < 150: b = beta

  Ipd[0]+=num_I
  
  return S,Epd,Ipd,Rpd, E, I, R

## Actual epi data from JHU

In [0]:
# Import confirmed cases
conf_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

#Import deaths data
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Import recovery data
rec_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [0]:
# Consolidate countries (ie. frenc dom tom are included in France, etc..)
conf_df = conf_df.groupby("Country/Region")
conf_df = conf_df.sum().reset_index()
conf_df = conf_df.set_index('Country/Region')

deaths_df = deaths_df.groupby("Country/Region")
deaths_df = deaths_df.sum().reset_index()
deaths_df = deaths_df.set_index('Country/Region')

rec_df = rec_df.groupby("Country/Region")
rec_df = rec_df.sum().reset_index()
rec_df = rec_df.set_index('Country/Region')

In [0]:
conf_df = conf_df.iloc[:,2:]
deaths_df = deaths_df.iloc[:,2:]
rec_df = rec_df.iloc[:,2:]

In [0]:
# Convert country names to correct format for search with pycountry
conf_df = conf_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})
# Convert country names to correct format for search with pycountry
deaths_df = deaths_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})
# Convert country names to correct format for search with pycountry
rec_df = rec_df.rename(index={'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo, the Democratic Republic of the', 'Burma': 'Myanmar', 'Korea, South': 'Korea, Republic of', 'Laos': "Lao People's Democratic Republic", 'Taiwan*': 'Taiwan', "West Bank and Gaza":"Palestine, State of"})

In [0]:
conf_df_pd = conf_df.diff(axis=1)
deaths_df_pd = deaths_df.diff(axis=1)
rec_df_pd = rec_df.diff(axis=1)

inf_df = conf_df - deaths_df - rec_df
inf_df_pd = inf_df.diff(axis=1)

In [0]:
fda100 = conf_df[conf_df > 100].apply(pd.Series.first_valid_index, axis=1)

In [0]:
#conf_df_pd[conf_df_pd.index=='Germany']

In [0]:
#conf_df_pd.loc['Germany',fda100['Germany']:].shape

In [14]:
fig = go.Figure(data=[
    go.Bar(name='I', x=np.arange(len(conf_df_pd.loc['Germany',fda100['Germany']:].index)), y=100*conf_df.loc['Germany',fda100['Germany']:].pct_change().values)
])
fig.show()

## Putting 2 and 2 together

In [0]:
fig = go.Figure(data=[
    go.Bar(name='I', x=np.arange(len(conf_df_pd.loc['Germany',:].index)), y=conf_df_pd.loc['Germany',:].values)
])

In [18]:
%prun model = SEIR_model(0.3, 10, 2, 5000, 100, len(conf_df_pd.loc['Germany',fda100['Germany']:]))

In [0]:
pd.Series(5000-model[0]).pct_change()

In [75]:
conf_df_pd.loc['Germany',fda100['Germany']:].values

array([  51.,   29.,   37.,   66.,  220.,  188.,  129.,  241.,  136.,
        281.,  451.,  170., 1597.,  910., 1210., 1477., 1985., 3070.,
       2993., 4528., 2365., 2660., 4183., 3930., 4337., 6615., 6933.,
       6824., 4400., 4790., 4923., 6064., 6922., 6365., 4933., 4031.,
       3251., 4289., 5633., 4885., 3990., 2737., 2946., 2218., 1287.,
       3394., 2945., 3699., 1945., 1842., 1881., 1226., 2357., 2481.])

In [19]:
fig.add_trace(go.Bar(name='Ipd of model', x=np.arange(len(model[0])), y=100*pd.Series(5000-model[0]).pct_change()))